In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display

from deep_orderbook.learn.test_learn import train_and_predict
from deep_orderbook.visu import Visualizer
from deep_orderbook.config import TrainConfig, ReplayConfig, ShaperConfig


vis = Visualizer()

train_config = TrainConfig()
replay_config = ReplayConfig(date_regexp='2024-08')
test_config = ReplayConfig(date_regexp='2024-09')
shaper_config = ShaperConfig(only_full_arrays=True)

# Define your asynchronous function to update the figure
async def online_learning(train_config, replay_config, shaper_config):
    async for books_arr, t2l, pxar, prediction, loss in train_and_predict(config=train_config, replay_config=replay_config, shaper_config=shaper_config, test_config=test_config):
        pred_t2l = prediction.reshape(t2l.shape).transpose(1, 0, 2)
        pred_t2l = np.clip(pred_t2l[:, :, 0], -1, 1)

        vis.add_loss(loss)
        vis.update(books_z_data=books_arr, level_reach_z_data=t2l, bidask=pxar, pred_t2l=pred_t2l)

await online_learning(train_config=train_config, replay_config=replay_config, shaper_config=shaper_config)